In [ ]:
# Installer nødvendige pakker før vi starter
# Legg merke til at alle de "vanlige" pakkene er forhåndsinstallerte
!pip install transformers pytorch-lightning sentence-transformers

In [ ]:
import pandas as pd
import numpy as np
from transformers import AutoModel, AutoTokenizer, AdamW, get_linear_schedule_with_warmup
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
from multiprocessing import cpu_count
from torch.cuda.amp import autocast
from torch import nn
import torch.nn.functional as F
import torch
from tqdm import tqdm_notebook
from sentence_transformers import SentenceTransformer, InputExample, SentencesDataset, losses, models
from sentence_transformers.evaluation import TripletEvaluator, SimilarityFunction
from sentence_transformers.util import pytorch_cos_sim

In [ ]:
# Google gir oss en tilfeldig GPU
# Typisk T4, V100, P100 eller K80
# T4 og V100 er svært raske kan kjøre 16bit operasjoner raskt
# K80 kan være håpløs trege for de tyngste nettverkene
torch.cuda.get_device_name(0)

# Config

# Dataset

In [ ]:
# Dere kan fikse datasett slik dere selv ønsker det, men her litt kode for å laste ned noen vanlige datasett

import os
import urllib.request
import logging
from functools import lru_cache
import zipfile
from glob import glob
import pandas as pd

log = logging.getLogger(__name__)


class EntityMatchingDataset:
    def __init__(self, name, data_dir = None):
        self.name = name
        if not data_dir:
            data_dir = os.path.join(os.getcwd(), "data")
        self.data_dir = data_dir
    
    @property
    @lru_cache(maxsize=1)
    def records_a(self):
        self.download()
        return self._load("records_a")
    
    @property
    @lru_cache(maxsize=1)
    def records_b(self):
        self.download()
        return self._load("records_b")
    
    @property
    @lru_cache(maxsize=1)
    def matches_train(self):
        self.download()
        return self._load("matches_train")
    
    @property
    @lru_cache(maxsize=1)
    def matches_val(self):
        self.download()
        return self._load("matches_val")
    
    @property
    @lru_cache(maxsize=1)
    def matches_test(self):
        self.download()
        return self._load("matches_test")
    
    def _download_file(self, url, filename):
        file_path = os.path.join(self.data_dir, self.name, filename)
        if not os.path.exists(file_path):
            log.warning(f"Downloading {url} to {file_path}")
            os.makedirs(os.path.dirname(file_path), exist_ok=True)
            urllib.request.urlretrieve(url, file_path)
    
    def download(self):
        ...
        
    def load(self):
        self.records_a
        self.records_b
        self.matches_train
        self.matches_val
        self.matches_test
        
class DeepMatcherDataset(EntityMatchingDataset):
    def __init__(self, name, url, data_dir = None):
        super().__init__(name, data_dir)
        self._url = url
    
    def download(self):
        self._download_file(urllib.parse.urljoin(self._url, "tableA.csv"), "tableA.csv")
        self._download_file(urllib.parse.urljoin(self._url, "tableB.csv"), "tableB.csv")
        self._download_file(urllib.parse.urljoin(self._url, "train.csv"), "train.csv")
        self._download_file(urllib.parse.urljoin(self._url, "valid.csv"), "valid.csv")
        self._download_file(urllib.parse.urljoin(self._url, "test.csv"), "test.csv")
    
    def _load(self, t):
        filename = {
            "records_a": "tableA.csv",
            "records_b": "tableB.csv",
            "matches_train": "train.csv",
            "matches_val": "valid.csv",
            "matches_test": "test.csv",
        }[t]
        if t.startswith("records"):
            return pd.read_csv(os.path.join(self.data_dir, self.name, filename), index_col="id").rename_axis(index="index")
        else:
            return pd.read_csv(os.path.join(self.data_dir, self.name, filename)).rename(columns={"ltable_id": "a.index", "rtable_id": "b.index", "label": "matching"}).astype({"matching": "bool"})

def deepmatcher_structured_amazon_google(data_dir=None):
    return DeepMatcherDataset("DeepMatcher/Structured/Amazon-Google", "http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/Amazon-Google/exp_data/", data_dir)
        
def deepmatcher_structured_beer(data_dir=None):
    return DeepMatcherDataset("DeepMatcher/Structured/Beer", "http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/Beer/exp_data/", data_dir)

def deepmatcher_structured_dblp_acm(data_dir=None):
    return DeepMatcherDataset("DeepMatcher/Structured/DBLP-ACM", "http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/DBLP-ACM/exp_data/", data_dir)

def deepmatcher_structured_dblp_google_scholar(data_dir=None):
    return DeepMatcherDataset("DeepMatcher/Structured/DBLP-GoogleScholar", "http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/DBLP-GoogleScholar/exp_data/", data_dir)
    
def deepmatcher_structured_fodors_zagats(data_dir=None):
    return DeepMatcherDataset("DeepMatcher/Structured/Fodors-Zagats", "http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/Fodors-Zagats/exp_data/", data_dir)
    
def deepmatcher_structured_walmart_amazon(data_dir=None):
    return DeepMatcherDataset("DeepMatcher/Structured/Walmart-Amazon", "http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/Walmart-Amazon/exp_data/", data_dir)
    
def deepmatcher_structured_itunes_amazon(data_dir=None):
    return DeepMatcherDataset("DeepMatcher/Structured/iTunes-Amazon", "http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/iTunes-Amazon/exp_data/", data_dir)

    
def deepmatcher_dirty_dblp_acm(data_dir=None):
    return DeepMatcherDataset("DeepMatcher/Dirty/DBLP-ACM" ,"http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Dirty/DBLP-ACM/exp_data/", data_dir)

def deepmatcher_dirty_dblp_google_scholar(data_dir=None):
    return DeepMatcherDataset("DeepMatcher/Dirty/DBLP-GoogleScholar", "http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Dirty/DBLP-GoogleScholar/exp_data/", data_dir)
    
def deepmatcher_dirty_walmart_amazon(data_dir=None):
    return DeepMatcherDataset("DeepMatcher/Dirty/Walmart-Amazon", "http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Dirty/Walmart-Amazon/exp_data/", data_dir)
    
def deepmatcher_dirty_itunes_amazon(data_dir=None):
    return DeepMatcherDataset("DeepMatcher/Dirty/iTunes-Amazon", "http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Dirty/iTunes-Amazon/exp_data/", data_dir)

    
def deepmatcher_textual_abt_buy(data_dir=None):
    return DeepMatcherDataset("DeepMatcher/Textual/Abt-Buy", "http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Textual/Abt-Buy/exp_data/", data_dir)
    
def deepmatcher_textual_company(data_dir=None):
    return DeepMatcherDataset("DeepMatcher/Textual/Company", "http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Textual/Company/exp_data/", data_dir)


class CompERBenchDataset(EntityMatchingDataset):
    def __init__(self, name, url, deduplication=False, data_dir = None):
        super().__init__(name, data_dir)
        self._url = url
        self._deduplication = deduplication
    
    def download(self):
        self._download_file(urllib.parse.urljoin(self._url, "records.zip"), "records.zip")
        if not os.path.exists(os.path.join(self.data_dir, self.name, "record_descriptions")):
            with zipfile.ZipFile(os.path.join(self.data_dir, self.name, "records.zip")) as zip_ref:
                zip_ref.extractall(os.path.join(self.data_dir, self.name))
        self._download_file(urllib.parse.urljoin(self._url, "gs_train.csv"), "gs_train.csv")
        self._download_file(urllib.parse.urljoin(self._url, "gs_val.csv"), "gs_val.csv")
        self._download_file(urllib.parse.urljoin(self._url, "gs_test.csv"), "gs_test.csv")
    
    def _load(self, t):
        path = {
            "records_a": glob(os.path.join(self.data_dir, self.name, "record_descriptions", "1_*"))[0],
            "records_b": glob(os.path.join(self.data_dir, self.name, "record_descriptions", "2_*" if not self._deduplication else "1_*"))[0],
            "matches_train": os.path.join(self.data_dir, self.name, "gs_train.csv"),
            "matches_val": os.path.join(self.data_dir, self.name, "gs_val.csv"),
            "matches_test": os.path.join(self.data_dir, self.name, "gs_test.csv"),
        }[t]
        if t.startswith("records"):
            return pd.read_csv(path, index_col="subject_id", encoding="iso-8859-1").rename_axis(index="index")
        else:
            return pd.read_csv(path, encoding="iso-8859-1").rename(columns={"source_id": "a.index", "target_id": "b.index"}).astype({"matching": bool})
        
def comperbench_abt_buy(data_dir=None):
    return CompERBenchDataset("CompERBench/abt-buy", "http://data.dws.informatik.uni-mannheim.de/benchmarkmatchingtasks/data/abt-buy/", data_dir)

def comperbench_wdc_xlarge_shoes(data_dir=None):
    return CompERBenchDataset("CompERBench/wdc_xlarge_shoes", "http://data.dws.informatik.uni-mannheim.de/benchmarkmatchingtasks/data/wdc_xlarge_shoes/", deduplication=True, data_dir=data_dir)

class LocalParquetDataset(EntityMatchingDataset):
    def __init__(self, name, data_dir = None):
        super().__init__(name, data_dir)
    
    def download(self):
        ...
        
    def _load(self, t):
        return pd.read_parquet(os.path.join(self.data_dir, self.name, t + ".parquet"))

In [ ]:
d = deepmatcher_textual_abt_buy()
d.load()

In [ ]:
# La oss se på noen av de positive matchene i treningsdataene
d.matches_train.query("matching").merge(d.records_a, left_on="a.index", right_index=True).merge(d.records_b, left_on="b.index", right_index=True)

In [ ]:
# Det er kjekt å ha enkelt tilgang til den formaterte streng-varianten av alle records senere
records_a = format_records(d.records_a)
records_b = format_records(d.records_b)

# Blocking

In [ ]:
# Veldig stor model
# bert_model = "roberta-base"

# En litt mindre modell som er ganske populær
#bert_model = "distilbert-base-cased"

# Google har en haug med modeller i alle tenkelige størrelser
# Her er noen i synkende størrelse - presisjonen faller hele veien
# De kan være fine å eksperimentere med noen ganger for å teste ting fortere
#bert_model = "google/bert_uncased_L-8_H-512_A-8"
bert_model = "google/bert_uncased_L-4_H-512_A-8"
#bert_model = "google/bert_uncased_L-4_H-256_A-4"
#bert_model = "google/bert_uncased_L-2_H-128_A-2"

# Det finnes ingen resultater som sier noe om ulike transformer-modeller for blocking.
# Vi tar en mindre en for blocking for å spare tid - egen erfaring tilsier at veldig store modeller gir mye mindre avkastning for blocking enn matching

### Data preprocessing and loading

In [ ]:
# For blocking trener vi ved hjelp av triplet loss.
# Altså istedenfor at loss'et reflekterer hvor like to embeddings skal være
# reflekterer det heller at vi vil at en embedding skal være mer lik en annen
# enn en tredje

def get_triplet_examples(records_a, records_b, matches):
    records_a = format_records(records_a)
    records_b = format_records(records_b)

    matches = matches.merge(records_a.to_frame("a.val"), left_on="a.index", right_index=True)
    matches = matches.merge(records_b.to_frame("b.val"), left_on="b.index", right_index=True)

    positives = matches[matches["matching"]]
    negatives = matches[~matches["matching"]]

    triplets_a = positives.merge(negatives[["a.index", "b.val"]].rename(columns={"b.val": "neg.val"}), on="a.index")[["a.val", "b.val", "neg.val"]]
    triplets_b = positives.merge(negatives[["b.index", "a.val"]].rename(columns={"a.val": "neg.val"}), on="b.index")[["b.val", "a.val", "neg.val"]]
    return list(InputExample(texts=t) for t in triplets_a.itertuples(index=False, name=None)) + list(InputExample(texts=t) for t in triplets_b.itertuples(index=False, name=None))

In [ ]:
input_examples = get_triplet_examples(d.records_a, d.records_b, d.matches_train)
val_input_examples = get_triplet_examples(d.records_a, d.records_b, d.matches_val)

train_dataset = SentencesDataset(input_examples, m)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=16)

### Model

In [ ]:
# Modellen vår produserer en embedding for et helt record vet å ta gjennomsnitt
# over embeddingene til alle tokens.
# SentenceTransformer-biblioteket tilbyr alt vi trenger.

word_embedding_model = models.Transformer(bert_model, max_seq_length=256)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

m = SentenceTransformer(modules=[word_embedding_model, pooling_model])

### Training

In [ ]:
# Det er masse hyperparametere som kan tunes, men disse er ganske gode default
# som jeg selv har funnet.
# Det meste relevante å endre er antall epochs.
# For vanskelige data eller hvis det er lite treningsdata kan man øke epochs noe,
# men hvis dataene er enkle og det er masse treningsdata kan man vurdere å senke
# antall epochs for å spare tid.

# Validerings-loss spyttes ut hver epoch - dette kan gi en indikasjon om nettverket overfitter. (Dette funker plutselig ikke?)

import logging
logging.getLogger().setLevel(logging.INFO) # Se validerings-loss

train_loss = losses.TripletLoss(m, distance_metric=losses.TripletDistanceMetric.COSINE, triplet_margin=0.1)
evaluator = TripletEvaluator([e.texts[0] for e in val_input_examples], [e.texts[1] for e in val_input_examples], [e.texts[2] for e in val_input_examples], main_distance_function=SimilarityFunction.COSINE)
m.fit(train_objectives=[(train_dataloader, train_loss)], epochs=3, warmup_steps=100, evaluator=evaluator, use_amp=True, evaluation_steps=len(train_dataloader))

### Testing

In [ ]:
# Vi starter med å finne embeddingen for alle records

embeddings_a = m.encode(records_a, batch_size=128)
embeddings_b = m.encode(records_b, batch_size=128)

In [ ]:
# La oss se hva recall og reduction rate blir hvis vi plukker ut de k nærmeste
# recordene for alle records i A og B

for k in [1, 5, 10, 20, 50, 100]:
  cos_scores = pytorch_cos_sim(embeddings_a, embeddings_b)
  cos_scores = cos_scores.cpu()

  blocked_matches = set()

  topk_sims, topk = torch.topk(cos_scores, k=k, dim=1)
  for i in range(topk.shape[0]):
    for j in range(topk.shape[1]):
      blocked_matches.add((i, topk[i, j].item()))

  topk_sims, topk = torch.topk(cos_scores, k=k, dim=0)
  for i in range(topk.shape[0]):
    for j in range(topk.shape[1]):
      blocked_matches.add((topk[i, j].item(), j))

  gold_matches = set(d.matches_test[d.matches_test["matching"]][["a.index", "b.index"]].itertuples(index=False, name=None))

  recall = len(gold_matches & blocked_matches) / len(gold_matches)
  reduction_rate = 1 - len(blocked_matches) / (len(records_a)*len(records_b))

  print(k, recall, reduction_rate)

# Matching

In [ ]:
# Veldig stor model - fungerer stort sett best blant modeller av tilsvarende størrelse og gir omtrentlig SOTA
# Større modeller finnes, men de blir gjerne håpløst uhåndterbare for forsvinnende liten gevinst
bert_model = "roberta-base"

# En litt mindre modell som er ganske populær
#bert_model = "distilbert-base-cased"

# Google har en haug med modeller i alle tenkelige størrelser
# Her er noen i synkende størrelse - presisjonen faller hele veien
# De kan være fine å eksperimentere med noen ganger for å teste ting fortere
#bert_model = "google/bert_uncased_L-8_H-512_A-8"
#bert_model = "google/bert_uncased_L-4_H-512_A-8"
#bert_model = "google/bert_uncased_L-4_H-256_A-4"
#bert_model = "google/bert_uncased_L-2_H-128_A-2"

### Data preprocessing and loading

In [ ]:
# Vi må konvertere records til et passende format for et transformer nettverk
# og sette opp nødvendig infrastruktur for å la PyTorch-verktøy laste inn
# batches med eksempler

# Vi gjør som Ditto og formaterer et record som:

# COL name VAL et navn COL description VAL en beskrivelse

def format_records(records):
    return records.apply(lambda r: " ".join(f"COL {c} VAL {v}" for c, v in r.iteritems()), axis=1)

class BertEntityMatchingDataset(Dataset):
    def __init__(self, records_a, records_b, matches, bert_model=None, tokenizer=None):
        assert bert_model is not None or tokenizer is not None

        self.records_a = records_a
        self.records_b = records_b
        self.matches = matches
        if tokenizer:
          self.tokenizer = tokenizer
        else:
          self.tokenizer = AutoTokenizer.from_pretrained(bert_model, use_fast=True)
        
        if "matching" in matches.columns:
          self._labels = matches["matching"]
        else:
          self._labels = None

        matches = matches[["a.index", "b.index"]]

        matches_a = matches.merge(records_a, how="left", left_on="a.index", right_index=True).drop(columns=["a.index", "b.index"])
        matches_b = matches.merge(records_b, how="left", left_on="b.index", right_index=True).drop(columns=["a.index", "b.index"])
        matches_a = format_records(matches_a).tolist()
        matches_b = format_records(matches_b).tolist()

        self._encoded_pairs = self.tokenizer(matches_a, matches_b, padding="max_length", truncation=True, max_length=512, return_tensors="pt")

    def __len__(self):
        return len(self.matches)
    
    def __getitem__(self, index):
        encoded_pair = {k: v[index] for k, v in self._encoded_pairs.items()}
        
        return encoded_pair, float(self._labels.iloc[index])


class BertEntityMatchingDataModule(pl.LightningDataModule):
    def __init__(self, dataset, bert_model, train_batch_size=64, inference_batch_size=128, num_workers=None):
        super().__init__()
        self._dataset = dataset
        self.bert_model = bert_model
        self.train_batch_size = train_batch_size
        self.inference_batch_size = inference_batch_size
        if num_workers is None:
            num_workers = cpu_count()
        self.num_workers = num_workers
    
    def prepare_data(self):
        self._dataset.download()
    
    def setup(self, stage = None):
        self._train = BertEntityMatchingDataset(self._dataset.records_a, self._dataset.records_b, self._dataset.matches_train, self.bert_model)
        self._val = BertEntityMatchingDataset(self._dataset.records_a, self._dataset.records_b, self._dataset.matches_val, self.bert_model)
        self._test = BertEntityMatchingDataset(self._dataset.records_a, self._dataset.records_b, self._dataset.matches_test, self.bert_model)
        
    def train_dataloader(self):
        return DataLoader(self._train, batch_size=self.train_batch_size, shuffle=True, num_workers=self.num_workers, pin_memory=True)
    
    def val_dataloader(self):
        return DataLoader(self._val, batch_size=self.inference_batch_size, num_workers=self.num_workers, pin_memory=True)
    
    def test_dataloader(self):
        return DataLoader(self._test, batch_size=self.inference_batch_size, num_workers=self.num_workers, pin_memory=True)

In [ ]:
data_module = BertEntityMatchingDataModule(
    d,
    bert_model,
    num_workers=0 if not torch.cuda.is_available() else None,
    train_batch_size=32,
    inference_batch_size=32
)
data_module.prepare_data()
data_module.setup()

### Model

In [ ]:
# Vi må definere nettverket vårt.
# Her er det gitt ved _Model, som legger til et enkelt lag på utsiden av
# et forhåndstrent transformer-nettverk

# I tillegg lager jeg her en Lightning-modul som sier hvordan vi ønsker å trene
# nettverket, validere og test.
# Man trenger ikke bruke PyTorch Ligthning, man kan også ganske enkelt skrive
# nødvendig logikk selv, slike moduler bare gjør det litt mer oversiktlig

class _Model(nn.Module):
    def __init__(self, bert_model):
        super().__init__()
        self.bert = AutoModel.from_pretrained(bert_model)
        self.dropout = nn.Dropout(0.1)
        self.cls_layer = nn.Linear(self.bert.config.hidden_size, 1)
    
    def forward(self, encoded_input):
        x = self.bert(**encoded_input)
        x = self.cls_layer(self.dropout(x[0][:, 0]))
        return x

class EntityMatchingModel(pl.LightningModule):
    def __init__(self, bert_model, lr=3e-5):
        super().__init__()
        self.save_hyperparameters()
        self.model = _Model(bert_model)

    def forward(self, x):
        return self.model(x).squeeze(dim=1)

    def training_step(self, batch, batch_idx):
        x, y = batch

        max_length = x["attention_mask"].detach().sum(axis=1).max().item()
        x = {k: v[:, :max_length] for k, v in x.items()}

        output = self(x)
        loss = nn.functional.binary_cross_entropy_with_logits(output, y)
        return loss

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=self.hparams.lr)
        data_loader = self.train_dataloader()
        steps = self.trainer.max_steps if self.trainer.max_steps else self.trainer.max_epochs*len(data_loader)
        lr_scheduler = {
            "scheduler": torch.optim.lr_scheduler.OneCycleLR(
                optimizer,
                max_lr=self.hparams.lr,
                total_steps=steps//self.trainer.accumulate_grad_batches, anneal_strategy="linear",
                div_factor=100,
                pct_start=0.05
            ),
            "interval": "step"
        }
        return [optimizer], [lr_scheduler]

    def _test_batch(self, batch):
        x, y = batch
        max_length = x["attention_mask"].detach().sum(axis=1).max().item()
        x = {k: v[:, :max_length] for k, v in x.items()}
        output = self(x)
        with autocast(enabled=False):
            y_hat = torch.sigmoid(output.type(torch.FloatTensor))
            loss = nn.functional.binary_cross_entropy(y_hat, y.type(torch.FloatTensor))
            predictions = torch.round(y_hat).type(torch.LongTensor)
            y = y.type(torch.LongTensor)
        return predictions, y, loss

    def _test_epoch(self, step_outputs):
        p = 0
        tp = 0
        fn = 0
        for pred, y in step_outputs:
            y = y.type(torch.LongTensor)
            p += pred.sum().item()
            tp += (y & pred).sum().item()
            fn += (y & (~pred)).sum().item()
        precision = tp / p if p > 0 else 0
        recall = tp / (tp + fn) if tp + fn > 0 else 0
        f1 = 2 * ((precision * recall) / (precision + recall)) if precision + recall > 0 else 0
        return precision, recall, f1

    def validation_step(self, batch, batch_idx):
        pred, y, loss = self._test_batch(batch)
        self.log('val_loss', loss, prog_bar=True)
        return pred, y

    def validation_epoch_end(self, validation_step_outputs):
        precision, recall, f1 = self._test_epoch(validation_step_outputs)
        self.log("val_prec", precision, prog_bar=True)
        self.log("val_recall", recall, prog_bar=True)
        self.log("val_f1", f1, prog_bar=True)

    def test_step(self, batch, batch_idx):
        pred, y, loss = self._test_batch(batch)
        self.log('test_loss', loss, prog_bar=True)
        return pred, y

    def test_epoch_end(self, test_step_outputs):
        precision, recall, f1 = self._test_epoch(test_step_outputs)
        self.log("test_prec", precision)
        self.log("test_recall", recall)
        self.log("test_f1", f1)

In [ ]:
m = EntityMatchingModel(bert_model)

### Training

In [ ]:
# Vi må trene modellen vår. Det er en haug med hyperparametere som kan stilles på,
# men disse er ganske gode defaults. Mest relevant er det kanskje å stille på
# antall epochs - noen ganger trenger man kanskje 5 eller 10.

# 16-bit presisjon er først og fremst for å utnytte muligheter for økt ytelse på
# moderne GPUer med støtte for raske 16-bit operasjoner.
# Det påvirker vanligvis ikke kvaliteten på resultatet fordi et smart bibliotek
# bak gardinene velger hvilke operasjoner som skal være 16-bit og ikke.

# Legg merke til at treningsrutinen spytter ut validerings-resultater etter hver epoch

trainer = pl.Trainer(gpus=1 if torch.cuda.is_available() else None, max_epochs=3, precision=16 if torch.cuda.is_available() else 32, checkpoint_callback=False, check_val_every_n_epoch=1)
trainer.fit(m, data_module)

### Testing

In [ ]:
# Evaluer på testdataene.
# Dette gjør du aldri underveis i utviklingen av metoden,
# bare som et aller siste trinn for å få de endelige resultatene
# å putte inn i masteroppgaven/paperet.
# Bruk valideringsdataene istedenfor underveis.
trainer.test(m, datamodule=data_module)

### Manual inference

Bare for å illustrere litt hva som skjer legger jeg til et eksempel på å
manuelt mate inn et par vi vil klassifisere

In [ ]:
# La oss ta en positive match fra valideringsdataene
record_pair = d.matches_val.query("matching").iloc[0]
record_pair

In [ ]:
record_a = records_a.loc[record_pair["a.index"]]
record_b = records_b.loc[record_pair["b.index"]]

print(record_a)
print(record_b)

In [ ]:
# Før vi mater det inn i nettverket må vi tokenize det på korrekt måte,
# pad'e korrekt før og mellom recordene og oversette hver token til en unik id

tokenizer = AutoTokenizer.from_pretrained(bert_model)
encoded_input = tokenizer([record_a], [record_b], return_tensors="pt")
encoded_input

In [ ]:
# Vi kan se på den tekstlige representasjonen av tokens
" ".join(tokenizer.convert_ids_to_tokens(encoded_input["input_ids"][0]))

In [ ]:
# Når vi gjør det manuelt må vi passe på å flytte all input til GPUen
encoded_input = {k: v.to("cuda") for k, v in encoded_input.items()}

torch.sigmoid(m(encoded_input))

### Save & Load

In [ ]:
# Det går an å lagre modellen til fil
# Dette er først og fremst aktuelt hvis du må restarte runtime (frivillig eller ufrivillig)
# så du slipper å trene igjen. Filer overlever runtime restart, men dette er ikke det samme som å avslutte sesjonen.
# Ikke sløs bort tid på å laste ned og opp modeller fra egen maskin mellom sesjoner.
# Filene er så store at det tar kortere tid å trene en modell igjen.
trainer.save_checkpoint("model.ckpt")

In [ ]:
# Laste modellen fra fil
m = EntityMatchingModel.load_from_checkpoint("model.ckpt", bert_model=bert_model, lr=3e-5)
m = m.to("cuda")